# Convert date to json file that can be used in visualisation

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import itertools
import json

**Load the data**

In [2]:
file_abg = Path('../Daten_Thurgau/Mitglieder_GR_2008_2021.csv')
file_vor = Path('../Daten_Thurgau/Vorstoesse_GR_ab_2012_Themen_def.csv')

In [3]:
vor = pd.read_csv(file_vor,delimiter=';')
abg = pd.read_csv(file_abg,delimiter=';')

**Nodes**

In [4]:
# select only the data after 2012
abg_2012 = abg.loc[abg['Jahr']>=2012]
vor_2012 = vor[vor.loc[vor['DATUM_VORSTOSS_EINGANG'].str[-4:] == '2012'].index[0]:]
# abg_list = pd.unique(abg_2012['Vorname'] + ' ' + abg_2012['Name'])
abg_list = abg_2012.drop(['Jahr','Geb_Jahr','Beruf','Fraktion','GR_Eintritt','Ort','Bezirk','Partei'],axis=1).drop_duplicates(ignore_index=True)
print(f'Wir haben {len(abg_list)} unabhängige Abgeordnete seit 2012') 

Wir haben 269 unabhängige Abgeordnete seit 2012


In [5]:
nodes = [dict(id=i,Name=abg_list['Vorname'].loc[i] + ' ' + abg_list['Name'].loc[i],Geschlecht=abg_list['Geschlecht'].loc[i]) for i in abg_list.index]
name_to_id = {n['Name']:n['id'] for n in nodes}

**Links**

In [15]:
# Auslesen der Unterzeichner:innen
links_list=None
vor_untz = vor_2012[[f'VORSTOESSER_IN_{i}' for i in range(1,11)]]
for i,vor_idx in enumerate(vor_untz.index):
    # Sortieren um jede Verbindung nur einmal zu haben (a,b) vs (b,a)
    vor_untz_i = sorted(list(vor_untz.loc[vor_idx].dropna()))
    # Erstellen einer lister aller Kombinationen
    vor_untz_i_links = list(itertools.combinations(vor_untz_i, 2))
    for l,link in enumerate(vor_untz_i_links):
        # convertin Names to ids
        link = [name_to_id[link[0]],name_to_id[link[1]]]
        # initialize links_list
        if links_list is None:
            links_list=[[link,[vor_idx]]]
        # add link if not present
        elif link not in [link[0] for link in links_list]:
            links_list.append([link,[vor_idx]])
        # if present append id of the vorstoss
        else:
            links_list[[link[0] for link in links_list].index(link)][1].append(vor_idx)

In [16]:
links = [dict(source=links_list[i][0][0],target=links_list[i][0][1],value=links_list[i][1]) for i in abg_list.index]

**Save json file**

In [22]:
netzwerk = dict(nodes=nodes,links=links)
with open("../Daten_Thurgau/netzwerk.json", "w") as file:
   json.dump(netzwerk, file)